In [1]:
import pandas as pd
import numpy as np


In [3]:
df = pd.read_csv("/Users/tanishsinha/Desktop/parkinsons-monitor/data/raw/parkinsons.csv")
df.head()


,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


In [4]:
df.columns


Index(['name', 'MDVP:Fo(Hz)', 'MDVP:Fhi(Hz)', 'MDVP:Flo(Hz)', 'MDVP:Jitter(%)',
       'MDVP:Jitter(Abs)', 'MDVP:RAP', 'MDVP:PPQ', 'Jitter:DDP',
       'MDVP:Shimmer', 'MDVP:Shimmer(dB)', 'Shimmer:APQ3', 'Shimmer:APQ5',
       'MDVP:APQ', 'Shimmer:DDA', 'NHR', 'HNR', 'status', 'RPDE', 'DFA',
       'spread1', 'spread2', 'D2', 'PPE'],
      dtype='object')

In [7]:
X = df.drop("status", axis=1)
y = df["status"]


In [8]:
X = X.drop("name", axis=1)


In [9]:
print(X.dtypes)
print(y.dtype)


MDVP:Fo(Hz)         float64
MDVP:Fhi(Hz)        float64
MDVP:Flo(Hz)        float64
MDVP:Jitter(%)      float64
MDVP:Jitter(Abs)    float64
MDVP:RAP            float64
MDVP:PPQ            float64
Jitter:DDP          float64
MDVP:Shimmer        float64
MDVP:Shimmer(dB)    float64
Shimmer:APQ3        float64
Shimmer:APQ5        float64
MDVP:APQ            float64
Shimmer:DDA         float64
NHR                 float64
HNR                 float64
RPDE                float64
DFA                 float64
spread1             float64
spread2             float64
D2                  float64
PPE                 float64
dtype: object
int64


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [11]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators=200,
    random_state=42
)

model.fit(X_train, y_train)


,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",200
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=""sqrt""The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to `""sqrt""`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",'sqrt'
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``, ``N_t_R`` and ``N_t_L`` all refer to the weighted sum,if ``sample_weight`` is passed... versionadded:: 0.19",0.0
,"bootstrap bootstrap: bool, default=TrueWhether bootstrap samples are used when building trees. If False, thewhole dataset is used to build each tree.",True
,"oob_score oob_score: bool or callable, default=FalseWhether to use out-of-bag samples to estimate the generalization score.By default, :func:`~sklearn.metrics.accuracy_score` is used.Provide a callable with signature `metric

In [12]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)


0.9230769230769231

In [13]:
risk_score = model.predict_proba(X_test)[:, 1]
risk_score[:10]


array([0.55 , 0.855, 0.985, 0.795, 0.165, 0.995, 0.645, 0.61 , 0.235,
       0.91 ])

In [14]:
def interpret_risk(score):
    if score < 0.3:
        return "Low Risk"
    elif score < 0.6:
        return "Moderate Risk"
    else:
        return "High Risk"

[interpret_risk(r) for r in risk_score[:10]]


['Moderate Risk',
 'High Risk',
 'High Risk',
 'High Risk',
 'Low Risk',
 'High Risk',
 'High Risk',
 'High Risk',
 'Low Risk',
 'High Risk']

In [15]:
import joblib

joblib.dump(model, "parkinsons_rf_model.pkl")


['parkinsons_rf_model.pkl']

In [16]:
import os
os.listdir()


['training.ipynb', 'parkinsons_rf_model.pkl', '.ipynb_checkpoints']

In [17]:
import joblib

model = joblib.load("parkinsons_rf_model.pkl")


In [18]:
joblib.dump(X.columns.tolist(), "model_features.pkl")


['model_features.pkl']

In [19]:
features = joblib.load("model_features.pkl")


In [20]:
loaded_model = joblib.load("parkinsons_rf_model.pkl")

loaded_model.predict_proba(X_test)[:5]


array([[0.45 , 0.55 ],
       [0.145, 0.855],
       [0.015, 0.985],
       [0.205, 0.795],
       [0.835, 0.165]])

In [21]:
from sklearn.metrics import accuracy_score

y_pred = loaded_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print("Accuracy:", acc)


Accuracy: 0.9230769230769231


In [22]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)


array([[ 8,  2],
       [ 1, 28]])

In [23]:
probs = loaded_model.predict_proba(X_test)[:, 1]

print("Min:", probs.min())
print("Max:", probs.max())
print("Mean:", probs.mean())


Min: 0.015
Max: 1.0
Mean: 0.689871794871795


In [24]:
sample = X_test.iloc[0]
sample_prob = loaded_model.predict_proba([sample])[0][1]

print("Parkinson's Risk:", sample_prob)


Parkinson's Risk: 0.55


/Users/tanishsinha/Desktop/parkinsons-monitor/venv/lib/python3.14/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [25]:
sample_df = X_test.iloc[[0]]   # double brackets → keeps column names

sample_prob = loaded_model.predict_proba(sample_df)[0][1]

print("Parkinson's Risk:", sample_prob)


Parkinson's Risk: 0.55


In [28]:
import joblib

# Save trained RandomForestClassifier
joblib.dump(model, "model/parkinsons_rf_model.pkl")

# Save the feature list used in training
joblib.dump(X_train.columns.tolist(), "model/model_features.pkl")

print("✅ Model and feature list saved successfully")


FileNotFoundError: [Errno 2] No such file or directory: 'model/parkinsons_rf_model.pkl'

In [29]:
import os

# Make 'model' folder if it doesn't exist
os.makedirs("model", exist_ok=True)
print("✅ 'model' folder is ready")


✅ 'model' folder is ready


In [30]:
import joblib

# Save trained RandomForestClassifier
joblib.dump(model, "model/parkinsons_rf_model.pkl")

# Save the feature list used in training
joblib.dump(X_train.columns.tolist(), "model/model_features.pkl")

print("✅ Model and feature list saved successfully")


✅ Model and feature list saved successfully


In [31]:
import joblib

# Assuming 'model' is your trained RandomForestClassifier
joblib.dump(model, "model/parkinsons_rf_model.pkl")


['model/parkinsons_rf_model.pkl']

In [32]:
import joblib

model = joblib.load("model/parkinsons_rf_model.pkl")
